In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [1]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

- Load data - just first 2 docs of training set
- text truncation
- load file with prompts


In [2]:
def text_truncation(tokens_list, maximum=500):
    selected_tokens = tokens_list[:maximum]  # Select the first 300 tokens
    return ' '.join(selected_tokens)  # Convert the list back to text

In [30]:
import pandas as pd
df = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")
print(list(set(df['label'])))
df = df.loc[df['set']=='train'].head(2)
df['text_trunc'] = df['tokens'].apply(text_truncation,100)
display(df)

import sys
sys.path.append('../scripts/') 
import prompt_template as pt


['Raadsnotulen', 'Factsheets', 'Actualiteit', 'Termijnagenda', 'Onderzoeksrapport', 'Motie', 'Raadsadres', 'Schriftelijke Vragen', 'Agenda', 'Besluit', 'Brief', 'Voordracht']


/tmp/ipykernel_61357/3685248844.py:5: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df['text_trunc'] = df['tokens'].apply(text_truncation,100)


,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages,clean_text,text_trunc
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,0,train,Gemeente Amsterdam\n% Gemeenteraad R\n% Gemeen...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",395,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",205,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,Gemeente Amsterdam % Gemeenteraad R % Gemeente...
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",390,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",197,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,Gemeente Amsterdam % Gemeenteraad R % Gemeente...


#### Tryout GEITje
Load chatbot

In [4]:
from transformers import pipeline, Conversation

chatbot = pipeline(task='conversational', model='Rijgersberg/GEITje-7B-chat-v2',
                   device_map='auto')

2024-04-03 10:39:13.377443: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-03 10:39:13.377478: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Simple query

In [ ]:
print(chatbot(
    Conversation('Welk woord hoort er niet in dit rijtje thuis: "auto, vliegtuig, geitje, bus"?')
))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Conversation id: c7f94172-c476-4fc4-a1cf-27485289b9d1
user: Welk woord hoort er niet in dit rijtje thuis: "auto, vliegtuig, geitje, bus"?
assistant: Geitje hoort er niet in thuis.



In [28]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_name = 'Rijgersberg/GEITje-7B-chat-v2'
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16,
                                             low_cpu_mem_usage=True, attn_implementation='eager',
                                             device_map=device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate(conversation, temperature=0.2, top_k=50, max_new_tokens=1_000):
    tokenized = tokenizer.apply_chat_template(conversation, add_generation_prompt=True,
                                              return_tensors='pt').to(device)
    outputs = model.generate(tokenized, do_sample=True, temperature=temperature,
                             top_k=top_k, max_new_tokens=max_new_tokens)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

conversation = [
    {
        'role': 'user',
        'content': 'Welk woord hoort er niet in dit rijtje thuis: "auto, vliegtuig, geitje, bus"?'
    }
]
print(generate(conversation))
# <|user|>
# Welk woord hoort er niet in dit rijtje thuis: "auto, vliegtuig, geitje, bus"? 
# <|assistant|>
# Het woord dat niet op zijn plaats staat is 'geit'. Een geit zou niet tussen een lijst van vervoersmiddelen moeten staan. Het past beter bij een boerderijthema of dierenlijst.

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 14.58 GiB of which 101.62 MiB is free. Including non-PyTorch memory, this process has 14.48 GiB memory in use. Of the allocated memory 14.33 GiB is allocated by PyTorch, and 19.38 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### Experiment set-up
Prompt GEITje for each document and save the prediction, return response, response time and the prompt version

In [146]:
from collections import Counter
import re
import time
import datetime
import pytz
import os
from sklearn.metrics import classification_report


# given the string response, extract the prediction
def get_prediction_from_response(response):
    # get a list of the possible classes
    classes_list = pt.get_class_list()

    predictions = [True if category.lower() in response.lower() else False for category in classes_list]

    # check if multiple classes were named, this is a prediction error
    if Counter(predictions)[True] > 1:
        return "PredictionError"

    # check if exactly one class is named, this is the prediction
    elif Counter(predictions)[True] == 1:
        prediction = [category.lower() for category in classes_list if category.lower() in response.lower()]
        return prediction[0]

    # if no class is named, then this is a no prediction error
    else:
        return 'NoPrediction'

# extract the promptfunction name
def get_promptfunction_name(prompt_function):
    string = f"{prompt_function}"
    match = re.search(r'<function\s+(\w+)', string)
    if match:
        function_name = match.group(1)
        return function_name
    else:
        return f"{prompt_function}"
    

def get_datetime():
    current_datetime_utc = datetime.datetime.now(pytz.utc)

    # Convert UTC time to Dutch time (CET)
    dutch_timezone = pytz.timezone('Europe/Amsterdam')
    current_datetime_dutch = current_datetime_utc.astimezone(dutch_timezone)
    return current_datetime_dutch
        

def get_runid(path):
    if os.path.exists(path):
        df = pd.read_pickle(path)
        return max(df['run_id'])+1, df

    else:
        return 0, pd.DataFrame()
    
def update_overview_results(df, model_name, subset=None):
 
    evaluation_dict = classification_report(df['label'], df['prediction'], output_dict=True)
    evaluation = pd.DataFrame(evaluation_dict).transpose()
    
    new_row = {
        'run_id':df.iloc[0]['run_id'],
        'model':model_name,
        'prompt_function':df.iloc[0]['prompt_function'],
        'text_column':df.iloc[0]['text_column'],
        'date': get_datetime(),
        'runtime':sum(df['runtime']),
        'set':subset,
        'support':evaluation.iloc[-1]['support'],
        'accuracy': evaluation_dict['accuracy'],

        'recall_weighted_avg':evaluation.loc[evaluation.index=='weighted avg']['recall'].values[0],
        'precision_weighted_avg': evaluation.loc[evaluation.index=='weighted avg']['precision'].values[0],
        'f1_weighted_avg': evaluation.loc[evaluation.index=='weighted avg']['f1-score'].values[0],

        'recall_macro_avg':evaluation.loc[evaluation.index=='macro avg']['recall'].values[0],
        'precision_macro_avg': evaluation.loc[evaluation.index=='macro avg']['precision'].values[0],
        'f1_macro_avg': evaluation.loc[evaluation.index=='macro avg']['f1-score'].values[0],


        'recall_classes': dict(zip(evaluation.index[0:-3], evaluation['recall'][0:-3])),
        'precision_classes': dict(zip(evaluation.index[0:-3], evaluation['precision'][0:-3])),
        'f1_classes': dict(zip(evaluation.index[0:-3], evaluation['f1-score'][0:-3])),
        'support_classes': dict(zip(evaluation.index[0:-3], evaluation['support'][0:-3])),

        'doc_paths':list(df['path'].values)
        
    }
    results = pd.DataFrame(columns=new_row.keys())
    results.loc[len(results)] = new_row
   
    path = f"{cf.output_path}/overview_results.pkl"
    if os.path.exists(path):
        earlier_results = pd.read_pickle(path)
        results = pd.concat([earlier_results, results])

    results.to_pickle(path)
   


In [139]:
update_overview_results(res, 'geitje')

/anaconda/envs/ThesisAmsterdamEnvironment19/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/anaconda/envs/ThesisAmsterdamEnvironment19/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/anaconda/envs/ThesisAmsterdamEnvironment19/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{

In [152]:
yeet = pd.read_pickle(f"{cf.output_path}/overview_results.pkl")
display(yeet)

yeet = pd.read_pickle(f"{cf.output_path}/predictions/ICgeitje_predictions.pkl")
display(yeet)


,run_id,model,prompt_function,text_column,date,runtime,set,support,accuracy,recall_weighted_avg,precision_weighted_avg,f1_weighted_avg,recall_macro_avg,precision_macro_avg,f1_macro_avg,recall_classes,precision_classes,f1_classes,support_classes,doc_paths
0,0,Rijgersberg/GEITje-7B-chat-v2,simple_prompt,text_trunc,2024-04-03 15:08:43.626104+02:00,18.881136,None,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'brief': 0.0, 'motie': 0.0, 'voordracht': 0.0}","{'brief': 0.0, 'motie': 0.0, 'voordracht': 0.0}","{'brief': 0.0, 'motie': 0.0, 'voordracht': 0.0}","{'brief': 0.0, 'motie': 2.0, 'voordracht': 0.0}",[/home/azureuser/cloudfiles/code/blobfuse/raad...
0,1,Rijgersberg/GEITje-7B-chat-v2,simple_prompt,text_trunc,2024-04-03 15:09:15.278413+02:00,18.851995,None,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'brief': 0.0, 'motie': 0.0, 'voordracht': 0.0}","{'brief': 0.0, 'motie': 0.0, 'voordracht': 0.0}","{'brief': 0.0, 'motie': 0.0, 'voordracht': 0.0}","{'brief': 0.0, 'motie': 2.0, 'voordracht': 0.0}",[/home/azureuser/cloudfiles/code/blobfuse/raad...


,id,path,text_column,prompt_function,response,prediction,label,runtime,date,run_id
0,0,/home/azureuser/cloudfiles/code/blobfuse/raads...,text_trunc,simple_prompt,Brief,brief,motie,7.803468,2024-04-03 15:08:32.485823+02:00,0
1,1,/home/azureuser/cloudfiles/code/blobfuse/raads...,text_trunc,simple_prompt,Voordracht,voordracht,motie,11.077668,2024-04-03 15:08:43.564827+02:00,0
0,0,/home/azureuser/cloudfiles/code/blobfuse/raads...,text_trunc,simple_prompt,Brief,brief,motie,7.768863,2024-04-03 15:09:04.137299+02:00,1
1,1,/home/azureuser/cloudfiles/code/blobfuse/raads...,text_trunc,simple_prompt,Voordracht,voordracht,motie,11.083132,2024-04-03 15:09:15.221748+02:00,1


In [151]:
import time
import datetime


""" Given a dataframe with txt, return a df with predictions """
# docs_df = dataframe with the documents that need to be predicted
# text_column = name of the column that includes the input_text. Can be different based on the text representation method. 
# prompt_function = prompt template -> ONLY prompt templates that take classes and doc as input (ZERO SHOT)

def zero_shot_predictions_incontextlearning(docs_df, text_column, prompt_function):
    results_df = pd.DataFrame(columns = ['id', 'path', 'text_column', 'prompt_function', 'response', 'prediction', 'label', 'runtime', 'date'])
    
    # prompt each document
    for index, row in docs_df.iterrows():
        start_time = time.time()

        # get the prompt, with the doc filled in
        txt = row[text_column]
        prompt = prompt_function(txt)

        # prompt and get the response
        converse = chatbot(Conversation(prompt))
        response = converse[1]['content']

        # extract prediction from response
        prediction = get_prediction_from_response(response)

        # save results in dataframe
        results_df.loc[len(results_df)] = {
            'id': row['id'],
            'path' : row['path'],
            'text_column' : text_column,
            'prompt_function': get_promptfunction_name(prompt_function),
            'response':response,
            'prediction':prediction,
            'label':row['label'].lower(),
            'runtime':time.time()-start_time,
            'date': get_datetime()
        }
    return results_df

""" Run a prediction function -> can be ZeroShot or FewShot """
def run_prediction(docs_df, text_column, prompt_function, subset=None, learning='ZeroShot'):
    if learning == 'ZeroShot':
        # get the predictions
        res = zero_shot_predictions_incontextlearning(docs_df, text_column, prompt_function)

        # get run_id
        path = f"{cf.output_path}/predictions/ICgeitje_predictions.pkl"
        res['run_id'], predictions_df = get_runid(path)

        all_predictions = pd.concat([predictions_df, res])

        # save predictions
        all_predictions.to_pickle(path)

        update_overview_results(res, 'Rijgersberg/GEITje-7B-chat-v2')
        return res



res = run_prediction(df, 'text_trunc', pt.simple_prompt)
display(res)


/anaconda/envs/ThesisAmsterdamEnvironment19/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/anaconda/envs/ThesisAmsterdamEnvironment19/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/anaconda/

,id,path,text_column,prompt_function,response,prediction,label,runtime,date,run_id
0,0,/home/azureuser/cloudfiles/code/blobfuse/raads...,text_trunc,simple_prompt,Brief,brief,motie,7.768863,2024-04-03 15:09:04.137299+02:00,1
1,1,/home/azureuser/cloudfiles/code/blobfuse/raads...,text_trunc,simple_prompt,Voordracht,voordracht,motie,11.083132,2024-04-03 15:09:15.221748+02:00,1
